# Create your own app using ipywidgets
<img src='https://images.unsplash.com/photo-1522926193341-e9ffd686c60f?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&auto=format&fit=crop&w=1650&q=80' width=400 height=400/>
<br>
<div class="list-group" id="toc" role="tablist">
  <h3 class="list-group-item list-group-item-action active" data-toggle="list"  role="tab" aria-controls="home">Table of Contents</h3>
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#1" role="tab" aria-controls="profile">Libraries Required<span class="badge badge-primary badge-pill">1</span></a>
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#2" role="tab" aria-controls="messages">Gathering the Data<span class="badge badge-primary badge-pill">2</span></a>
  <a class="list-group-item list-group-item-action"  data-toggle="list" href="#3" role="tab" aria-controls="settings">Designing the Components<span class="badge badge-primary badge-pill">3</span></a>
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#3.1" role="tab" aria-controls="settings">->Species, Recordist & Audio options<span class="badge badge-primary badge-pill">3.1</span></a> 
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#3.2" role="tab" aria-controls="settings">-> Title + Audio Container<span class="badge badge-primary badge-pill">3.2</span></a>
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#3.3" role="tab" aria-controls="settings">-> Image + Description Container<span class="badge badge-primary badge-pill">3.3</span></a>
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#3.4" role="tab" aria-controls="settings">-> Bird Stats<span class="badge badge-primary badge-pill">3.4</span></a>
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#3.5" role="tab" aria-controls="settings">-> Waveform Plot<span class="badge badge-primary badge-pill">3.5</span></a>
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#3.6" role="tab" aria-controls="settings">-> Map Plot<span class="badge badge-primary badge-pill">3.6</span></a>
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#3.7" role="tab" aria-controls="settings">-> Combining all components<span class="badge badge-primary badge-pill">3.7</span></a>
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#4" role="tab" aria-controls="settings">App Interactions<span class="badge badge-primary badge-pill">4</span></a>
</div>

Here's an working example of the App.

<img src='https://media.giphy.com/media/kBptuCNtycKSP4UiWg/giphy.gif'>

# <a id='1'>1. Libraries Required</a>
<a href='#toc'><span class="label label-info">Go back to the Table of Contents</span></a>

Let's first install all the libraries required for the application.

In [ ]:
!pip install pydub

In [ ]:
#Libraries Required
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import os
import requests
import re
import IPython.display as ipd
from pydub import AudioSegment
import requests
import json
from bs4 import BeautifulSoup
import ipywidgets as widgets

# <a id='2'>2. Gathering Data</a>
<a href='#toc'><span class="label label-info">Go back to the Table of Contents</span></a>

I am using the [public dataset](https://www.kaggle.com/anshuls235/birdsongrecognitiondetails) uploaded by me directly.

But, if you want to know the code to get the bird details refer to the below commented code. I have scrapped these details using beautiful soup and requests library. The webpage from where I have scraped the data is https://ebird.org/species.

In [ ]:
"""
dic_bird = {} 
for code in df_train.ebird_code.unique():
    if code not in dic_bird:
        url = 'https://ebird.org/species/'+code
        r = requests.get(url)
        soup = BeautifulSoup(r.content, 'html.parser')
        desc = soup.find('meta',property="og:description")
        image = soup.find('meta',property="og:image")
        dic_bird[code]={}
        dic_bird[code]['description'] = desc['content']
        dic_bird[code]['image'] = image['content']
#Convert the dictionary to dataframe.
df = pd.DataFrame(columns=['species','description','image'])
i = 0
for key,val in dic_bird.items():
    df.loc[i,'species'] = key
    df.loc[i,'description'] = val['description']
    df.loc[i,'image'] = val['image']
    i+=1
df.to_csv('bird_details.csv',index=False)
"""

As can be seen from the code, to get the details of a particular species of the bird we need to append its `ebird_code` to the site URL.

In [ ]:
#Get the training data and the dataset containing images & descriptions
df_train = pd.read_csv('/kaggle/input/birdsong-recognition/train.csv')
df = pd.read_csv('/kaggle/input/birdsongrecognitiondetails/bird_details.csv')
media_path = '/kaggle/input/birdsong-recognition/train_audio/'

# <a id='3'>3. Designing the Components</a>
<a href='#toc'><span class="label label-info">Go back to the Table of Contents</span></a>

After collecting all the data, the next logical step is to create the components of the app one by one. I’ll be listing down the steps for all components one by one in sequence.

## <a id='3.1'>3.1 Species, Recordist & Audio options</a>
<img src='https://miro.medium.com/max/1400/1*TYLwQNGH4_zDNrAA9F9h0A.png'/>

This is an example of how it will look. The user can easily scroll through all the options easily. Also, the values of recordists and filename change on the basis of the species selected. This will be covered in the interactions section later. The code to implement this is as follows:

In [ ]:
# Dictionary to Store Default values
default_values = {
    'species': 'Alder Flycatcher',
    'recordist': 'Jonathon Jongsma',
    'audio': 'XC134874.mp3',
    'image': df.loc[df['species']=='aldfly','image'].tolist()[0],
    'text': df.loc[df['species']=='aldfly','description'].tolist()[0],
    'filename': os.path.join('/kaggle/input/birdsong-recognition/train_audio/','aldfly','XC134874.mp3'),
    'latitude': 44.793,
    'longitude': -92.962,
    'location': 'Grey Cloud Dunes SNA, Washington, Minnesota'
}

def get_recordist_options(species):
    """Getter method for Recordist values."""
    return df_train[df_train['species']==species]['author'].unique().tolist()

def get_audio_options(species,recordist):
    """Getter method for Audio file values."""
    return df_train[(df_train['species']==species)&(df_train['author']==recordist)]['filename'].unique().tolist()

species = widgets.Dropdown(
    description = 'Species:   ',
    value = default_values['species'],
    options = df_train['species'].unique().tolist(),
    layout=dict(width='233px')
)

recordists = widgets.Dropdown(
    description = 'Recordist:   ',
    value = default_values['recordist'],
    options = get_recordist_options(default_values['species']),
    layout=dict(width='233px')
)

audios = widgets.Dropdown(
    description = 'Filename:    ',
    value = default_values['audio'],
    options = get_audio_options(default_values['species'],default_values['recordist']),
    layout=dict(width='233px')
)
container1 = widgets.HBox(children=[species, recordists, audios])
container1

## <a id='3.2'>3.2 Title + Audio Container</a>
<a href='#toc'><span class="label label-info">Go back to the Table of Contents</span></a>

<img src='https://miro.medium.com/max/1400/1*F8xYmRPsh-Z7aAtbAwXi_w.png'/>
The title can be easily added using the HTML widgets. It changes depending on the species selected. The audio is shown using the Audio widget.

In [ ]:
title = widgets.HTML('<h1>{}</h1>'.format(default_values['species']),layout=dict(width='350px'))
out = widgets.Output(layout=dict(width='350px',margin='10px 0px 0px 0px'))
out.append_display_data(ipd.Audio(default_values['filename']))
container2 = widgets.HBox(children=[title,out])
container2

## <a id='3.3'>3.3 Image + Description Container</a>
<a href='#toc'><span class="label label-info">Go back to the Table of Contents</span></a>

<img src='https://miro.medium.com/max/1400/1*Whlb32fpXU-KlqQaEEyTQA.png'/>
I have added image and description using the HTML widget. Later, both are pooled into a Horizontal Box container using HBox widget. Also, layout can be defined for each widget separately. This is exactly similar to CSS.

In [ ]:
im = widgets.HTML('<img src="{}"/>'.format(default_values['image']),
                 layout=dict(height='250px',width='300px'))
text = widgets.HTML('<h5>{}</h5>'.format(default_values['text']),
                 layout=dict(height='250px',width='400px',margin='10px 0px 0px 10px'))
container3 = widgets.HBox(children=[im,text])
container3

## <a id='3.4'>3.4 Statistics about birds</a>
<a href='#toc'><span class="label label-info">Go back to the Table of Contents</span></a>

<img src='https://miro.medium.com/max/1400/1*QWiRy5YqYYMzAhEJTb-RnQ.png'/>
Avg. Rating is the mean rating of all the ratings for short recordings of bird’s species calls generously uploaded by users of xenocanto.org. Average duration is the mean duration of the bird calls. Average elevation as the name suggests is the mean height the bird travels at.
<br><br>
Elevation is provided to us in a string format with lot of variations. In the below code block I've added the function to derive the elevation of the bird.

In [ ]:
def get_elevation(val):
    """Derive the elevation value from the string. Also, I have 
    kept negative elevation values as below sea level is also a possibility."""
    l = re.findall('[~\?]?(-?\d+[\.,]?\d*)-?(\d*)',val)
    val1=0
    val2=0
    if l:
        if l[0][0]:
            val1=float(l[0][0].replace(',',''))
        if l[0][1]:
            val2=float(l[0][1].replace(',',''))
        if val1!=0 and val2!=0:
            return (val1+val2)/2
        return val1
    else:
        return float('nan')
df_train.elevation=df_train.elevation.apply(lambda x: get_elevation(x))

def get_stats(species):
    """Get the Average rating,Duration of chip &
        elevation of the bird species."""
    df_sp = df_train[df_train['species']==species]
    avg_rating = np.round(df_sp.rating.mean(),2)
    avg_duration = np.round(df_sp.duration.mean(),2)
    avg_elevation = np.round(df_sp.elevation.mean(),2)
    return avg_rating,avg_duration,avg_elevation

r,d,e = get_stats(default_values['species'])
rating = widgets.HTML('<h2>Avg. Rating</h2><h4>{}</h4>'.format(r),layout=dict(width='233px'))
duration = widgets.HTML('<h2>Avg. Duration</h2><h4>{} s</h4>'.format(d),layout=dict(width='233px'))
elevation = widgets.HTML('<h2>Avg. Elevation</h2><h4>{} m</h4>'.format(e),layout=dict(width='233px'))
container4 = widgets.HBox(children=[rating,duration,elevation])
container4

## <a id='3.5'>3.5 Waveform Plot</a>
<a href='#toc'><span class="label label-info">Go back to the Table of Contents</span></a>

<img src='https://miro.medium.com/max/1400/1*Kw49sRLGC7-Wm3XRVaJx2g.png'/>
To show bird chirp graphically I'll be using a waveform plot. I have used a helper function to convert mp3 to NumPy array and then used Plolty FigureWidget to create the waveform plot. Simple matplotlib plots can be used as well but I always prefer Plotly owing to its added interactivity. The waveform changes on the basis of the audio file selected.

The audios in our dataset can be mono or stereo. So, we need to take care of the number of channels. I have used plotly's [FigureWidget](https://plotly.com/python/figurewidget/) as it works pretty well with ipywidgets. 

In [ ]:
def read(f, normalized=False):
    """Converts MP3 to numpy array"""
    a = AudioSegment.from_mp3(f)
    y = np.array(a.get_array_of_samples())
    if a.channels == 2:
        y = y.reshape((-1, 2))
    if normalized:
        return a.frame_rate, np.float32(y) / 2**15
    else:
        return a.frame_rate, y

def plot_waveform(arr,filename):
    """Plots the waveform from the numpy array"""
    fig = go.FigureWidget()
    try:
        channels = arr.shape[1]
        for channel in range(channels):
            fig.add_trace(go.Scatter(name='channel '+str(channel+1),y=arr[:,channel]))
    except IndexError:
        fig.add_trace(go.Scatter(y=arr,showlegend=False))
    fig.update_layout(template='seaborn',plot_bgcolor='rgb(255,255,255)',paper_bgcolor='rgb(255,255,255)',
                 height = 200, width = 700,title=filename,legend=dict(x=0.3,y=1.3,orientation='h'),
                 xaxis=dict(mirror=True,linewidth=2,linecolor='black'),
                 yaxis=dict(mirror=True,linewidth=2,linecolor='black'),
                 margin=dict(l=0,r=0,t=0,b=5))
    return fig
    
rate, arr = read(default_values['filename'])
title_g = widgets.HTML('<h2>Waveform of the bird chirp</h2>')
g = plot_waveform(arr,default_values['filename'].split('/')[-1])

## <a id='3.6'>3.6 Map plot</a>
<a href='#toc'><span class="label label-info">Go back to the Table of Contents</span></a>

<img src='https://miro.medium.com/max/1400/1*_gPQ-yWnbEcAmhqWnvGcSA.png'/>
I have used Plotly’s scattermapbox to plot the recording location on the map. It also changes depending on the audio file selected.
To plot the location on map, where a particular recording was made, we would require the below method. It takes in the latitude, longitude and the location from the train dataset and plots it on a mapbox plot.
> **Please Note:** The dimensions of the plot are as per the app. So, you can change it acoordingly in the function.

In [ ]:
#get the mapbox token
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_1 = user_secrets.get_secret("mapboxtoken")

def plot_location(lat,lon,location):
    """Plots the recording location on the map."""
    trace = go.Scattermapbox(lon=[lon],lat=[lat],hovertext=location,
                                           marker=dict(symbol='campsite',size=20,color='blue'))
    data = [trace]
    layout = go.Layout(
        width=700,
        height=200,
        margin=dict(l=0,r=0,t=5,b=0),
        hovermode='closest',
        mapbox=dict(
            accesstoken=secret_value_1,
            bearing=0,
            style='satellite-streets',
            center=go.layout.mapbox.Center(
                lat=lat,
                lon=lon
            ),
            pitch=0,
            zoom=10
        )
    )
    figure = go.Figure(data=data, layout=layout)
    fig = go.FigureWidget(figure)
    return fig

title_map = widgets.HTML('<h2>Where the recording was made?</h2>')
loc_map = plot_location(default_values['latitude'],default_values['longitude'],default_values['location']) 

**Please note:** In case you are using plotly for your plots, all the plots should be added to the plotly.graph_objects.Figurewidget for ipywidgets to work properly with them.

## <a id='3.7'>3.7 Combining all Components</a>
<a href='#toc'><span class="label label-info">Go back to the Table of Contents</span></a>

After designing all the components, just pool them into a single component.

In [ ]:
app = widgets.VBox(children=[container1,container2,container3,container4,title_g,g,title_map,loc_map])

# <a id='4'>4. App Interactions</a>
<a href='#toc'><span class="label label-info">Go back to the Table of Contents</span></a>

Following all the steps in section 2 will produce a static app. But, Nobody likes a static app! So, to add interactions to our ipywidgets app we need to add callback functions which will take care of changing the data of the components we designed. The following are the interactions which take place in the app:
- `change species`-> change recordists dropdown, change audios dropdown, change title, image & text, and change bird stats.
- `change recordist`-> change audios dropdown.
- `change audio`-> change waveform and recording location map.

I have kept separate callbacks to stick to the flow listed above. Please note when `response_sp()` callback is triggered it will automatically trigger `response_re()` and `response_au()` callbacks respectively as recordists and audio values are changing in the species change callback function.
Similarly, `response_re()` will trigger `response_au()`. And `response_au()` alone won’t trigger any callbacks.

In [ ]:
def get_filename(ebird_code,file):
    """Getter method for filename."""
    return os.path.join('/kaggle/input/birdsong-recognition/train_audio/',ebird_code,file)

def get_ebird_code(species):
    """Getter method for ebird_code."""
    return df_train.loc[df_train['species']==species,'ebird_code'].tolist()[0]

def get_image(ebird_code):
    """Getter method for image URL."""
    return df.loc[df['species']==ebird_code,'image'].tolist()[0]

def get_text(ebird_code):
    """Getter method for species description."""
    return df.loc[df['species']==ebird_code,'description'].tolist()[0]

def response_sp(change):
    """callback function for species dropdown"""
    ecode = get_ebird_code(species.value)
    
    #change recordists dropdown
    options = get_recordist_options(species.value)
    recordists.options = options
    recordists.value = options[0]
    
    #change filename dropdown
    options = get_audio_options(species.value,recordists.value)
    audios.options = options
    audios.value = options[0]
    
    #change title, image and text
    title.value = '<h1>{}</h1>'.format(species.value)
    im.value = '<img src="{}"/>'.format(get_image(ecode))
    text.value = '<h5>{}</h5>'.format(get_text(ecode))
    
    #change bird stats
    r,d,e = get_stats(species.value)
    rating.value = '<h2>Avg. Rating</h2><h4>{}</h4>'.format(r)
    duration.value = '<h2>Avg. Duration</h2><h4>{} s</h4>'.format(d)
    elevation.value = '<h2>Avg. Elevation</h2><h4>{} m</h4>'.format(e)

def response_re(change):
    """callback function for recordists dropdown."""
    #change audios dropdown
    options = get_audio_options(species.value,recordists.value)
    audios.options = options
    audios.value = options[0]

def response_au(change):
    """callback function for audios dropdown."""
    ecode = get_ebird_code(species.value)
    file = audios.value
    filename = get_filename(ecode,file)
    with out:
        ipd.clear_output()
        ipd.display(ipd.Audio(filename))
    rate, arr = read(filename)
    g_new = plot_waveform(arr,filename.split('/')[-1])
    g.update(data=g_new.data,layout=g_new.layout)
    lat = df_train.loc[df_train['filename']==audios.value,'latitude'].tolist()[0]
    lon = df_train.loc[df_train['filename']==audios.value,'longitude'].tolist()[0]
    location = df_train.loc[df_train['filename']==audios.value,'location'].tolist()[0]
    loc_map_new = plot_location(float(lat),float(lon),location)
    loc_map.update(data=loc_map_new.data,layout=loc_map_new.layout)

#Definition of callbacks    
species.observe(response_sp, names="value")
recordists.observe(response_re, names="value")
audios.observe(response_au, names="value")

### Finally the app is complete!
Just run the below code cell now to see the entire app in action.(Provided you have executed all the steps above.)

In [ ]:
#Run the App!
app

### It's dissapointing that kaggle notebooks don't support ipywidgets currently(On commit only). If you fork the kernel and then execute the app it will run successfully and you play around. But, doesn't work on commit. Hopefully the Kaggle team addresses this issue.

# Thanks for reading this far. Do leave an upvote in case you liked my work. It motivates me to produce more quality content.
### All the steps along with explanations are also available in my [medium article](https://medium.com/analytics-vidhya/diy-interactive-app-just-using-python-55134429bf35).
### [Click here](https://www.kaggle.com/anshuls235/notebooks) to view my other kernels.
<img src='https://media.giphy.com/media/MCupMc20PsFIct6n1G/giphy.gif'/>